# 1. Norsk UD-trebank: Et nynorsk tekstdatasett

# Sammendrag


Den norske UD-trebanken er basert på nynorskdelen av den norske
Dependency Treebank (NDT), som er en norsk, syntaktisk trebank.
NDT er automatisk konvertert til UD-schema av Lilja Øvrelid ved Universitetet i Oslo.


# Introduksjon


NDT ble utviklet fra 2011 til 2014 ved Nasjonalbiblioteket i samarbeid med Tekstlaboratoriet og Institutt for informatikk ved Universitetet i Oslo.
NDT inneholder rundt 300 000 tokens hentet fra en rekke forskjellige sjangre. Trebanktekstene er manuelt annotert for morfosyntaktisk informasjon. Denne annoteringen følger hovedsakelig [Oslo-Bergen Tagger](http://tekstlab.uio.no/obt-ny/). Den syntaktiske annoteringen følger i stor grad Norwegian Reference Grammar, samt et avhengighetsmerkeskjema formulert ved begynnelsen av annoteringsprosjektet og iterativt raffinert gjennom byggingen av trebanken. For mer informasjon se referanser nedenfor.

## Kjør koden nedenfor for å laste inn datasettet:

In [ ]:
from utils import read_data

nynorskCorpus, nynorskTokens = read_data("../datasets/UD_Norwegian-Nynorsk-master/no_nynorsk-ud-train.conllu")
# nynorskCorpus, nynorskTokens = read_data("../datasets/UD_Norwegian-Nynorsk-master/no_nynorsk-ud-test.conllu")

## 1.1 Visualiser tekstkorpuset

La oss se nærmere på ordene som er tilgjengelige i datasettet.

### Ordsky

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(nynorskTokens))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud")
plt.show()

### Ordfrekvens

La oss undersøke hvilke ord som er mest vanlige i datasettet

In [ ]:
import pandas as pd
from nltk import FreqDist
import seaborn as sns

freq_dist = FreqDist(nynorskTokens)
df_freq_dist = pd.DataFrame(list(freq_dist.items()), columns=["Token", "Frequency"])
df_freq_dist = df_freq_dist.sort_values(by="Frequency", ascending=False)
plt.figure(figsize=(12, 6))
sns.barplot(x="Token", y="Frequency", data=df_freq_dist.head(20))
plt.title("Top 20 Most Frequent Tokens")
plt.xticks(rotation=45, ha="right")
plt.show()

### Ordlengde

Hvor lange er ordene i datasettet?

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(range(len(nynorskTokens)), [len(token) for token in nynorskTokens], alpha=0.5)
plt.title("Scatter Plot of Word Lengths")
plt.xlabel("Token Index")
plt.ylabel("Word Length")
plt.show()

# 2. Tren en enkel språkmodell for setningsgenerering

<div style="display: flex; flex-direction: row;">
    <img src="resources/ngram.png" alt="Image 1" width="500px" height="150px" style="margin-left: 300px;">
</div>

N-gram-modeller er grunnleggende språkmodeller for tekstgenerering, og gir et sannsynlighets-rammeverk for å beskrive strukturen og mønstrene i en sekvens av ord. I kontekst av NLP er en n-gram en sekvens av n ord, og n-gram-modeller estimerer sannsynligheten for et ord basert på dets kontekst – de foregående n-1 ordene. Antakelsen er at sannsynligheten for et ord bare avhenger av en begrenset historie med foregående ord, noe som gjør beregningen mer gjennomførbar. Disse modellene tilbyr enkelhet og effektivitet, noe som gjør dem grunnleggende i ulike språkbehandlingsoppgaver inkludert tekstgenerering, maskinoversettelse og talegjenkjenning. N-gram-modeller viser effektivitet når det gjelder lokale avhengigheter, men de kan sliter over lengre rekkevidde og mislykkes i å fange opp den bredere semantiske konteksten som finnes i mer avanserte modeller.

In [ ]:
# train a simple Bigram model from the training data loaded above.

from utils import train_ngram_model
ngram_model = train_ngram_model(nynorskTokens)

In [ ]:
# Generate text starting with a seed
from utils import generate_text

seed = '...' # The initial prompt for generating a sentence.
generated_text = generate_text(ngram_model , seed, length=50) # generate sample text using the given function.
print(generated_text)

# 3. Store språkmodeller

<div style="display: flex; flex-direction: row;">
    <img src="resources/lstm.png" alt="Image 1" width="500px" height="150px" style="margin-right: 10px;">
    <img src="resources/transformer.png" alt="Image 2" width="500px" height="100" style="margin-right: 10px;">
</div>

Språkmodeller basert på nevrale nettverk, slik som recurrent neural networks (RNNs), long short-term memory networks (LSTMs) og transformerarkitekturer, har vist overlegen resultater sammenlignet med n-gram-modeller. Her er noen grunner til at nevrale språkmodeller generelt anses som bedre:

***Avhengigheter:***

N-gram-modeller fanger avhengigheter opp til et fast antall foregående ord (n i n-gram). Nevrale språkmodeller, spesielt transformerarkitekturer, kan fange opp avhengigheter over en lengre rekkefølge av ord, noe som gjør at de kan modellere mer komplekse sammenhenger.

***Parameter-effektivitet:***

Nevrale språkmodeller kan effektivt representere og lære fra store mengder data med relativt få parametere sammenlignet med n-gram-modeller. Dette er avgjørende for å håndtere den store mengden informasjon som finnes i naturlig språk.

***Kontinuerlig embeddings:***

Nevrale modeller representerer ord som kontinuerlige embeddings i et høydimensjonalt rom. Denne kontinuerlige representasjonen gjør at modellen kan fange opp semantiske forhold mellom ord, noe som er utfordrende for diskrete representasjoner brukt i n-gram.

***Generalisering:***

Nevrale modeller generaliserer bedre til ukjente eller sjeldne ord fordi de lærer kontinuerlige representasjoner som fanger opp likheter mellom ord. N-gram-modeller sliter med ord som ikke finnes i vokabularet og generaliserer dårlig til ukjente kontekster.

***Tilpasningsevne:***

Nevrale modeller kan tilpasse seg kompleksiteten til forskjellige NLP-oppgaver ved å justere hyperparametere. N-gram-modeller har begrenset kapasitet til å tilpasse seg forskjellige oppgaver uten å endre n-gram-rekkefølgen, noe som ikke er praktisk.

***Variabel kontekstlengde:***

Nevrale modeller kan håndtere kontekster av variablel lengde, noe som gjør dem mer fleksible i behandlingen av sekvenser med forskjellige lengder. I motsetning til dette krever n-gram-modeller kontekster med fast lengde, noe som kan være begrensende.

***Kontekstuell informasjon:***

Nevrale modeller som BERT (Bidirectional Encoder Representations from Transformers) og GPT (Generative Pre-trained Transformer) tar hensyn til kontekstuell informasjon ved å behandle hele sekvensen bidireksjonalt eller unidireksjonalt. Dette gjør at de kan fange opp rikere kontekst for hvert ord.

***Ytelse:***

Nevrale språkmodeller har oppnådd state-of-the-art ytelse på ulike NLP-benchmarktester, inkludert oppgaver som språkmodellering, maskinoversettelse, tekstsummering og sentimentanalyse. Til tross for fordelene med nevrale språkmodeller, kan n-gram-modeller fortsatt være nyttige i visse scenarier, spesielt når en har begrensede ressurser eller når en enkel modell er tilstrekkelig for oppgaven. Valget mellom n-gram-modeller og nevrale språkmodeller avhenger ofte av de spesifikke kravene til oppgaven, mengden tilgjengelige data og ressurser.



## Tokenisering

Sjekk ut denne lenken som viser tokenisering for ChatGPT: https://platform.openai.com/tokenizer

<div style="display: flex; flex-direction: row;">
    <img src="resources/tokenizer.png" alt="Image 1" width="700px" height="300px" style="margin-left: 200px;">
</div>

In [ ]:
from utils import tokenize_text
tokens = tokenize_text("Supercalifragilisticexpialidocious")
print(tokens)


tokens = tokenize_text("ordbokstavrimkonkurranse")
print(tokens)


tokens = tokenize_text("Enter your word here")
print(tokens)

## Å kjøre en LLM lokalt  er ressurskrevende og tar lang tid. Vi vil derfor i neste avsnitt bruke OpenAIs API for ekstern tilgang til en GPT-modell.